In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def load_data(file_path):
    df = pd.read_csv(file_path)
    df['model'] = df['model'].str.replace('Prophet_', '')
    return df

# Função para plotar o heatmap das métricas
def plot_metrics_heatmap(df, highlight_model):
    metrics = ['mae', 'mape', 'smape', 'rmse', 'training_time']
    plt.figure(figsize=(12, 10))
    sns.heatmap(df[metrics], annot=True, cmap='YlOrRd', fmt='.4f')
    plt.title('Heatmap das Métricas de Avaliação')
    
    # Destacar o modelo específico
    highlight_index = df[df['covariates'] == highlight_model].index[0]
    plt.axhline(y=highlight_index, color='blue', linewidth=2)
    plt.axhline(y=highlight_index+1, color='blue', linewidth=2)
    
    plt.tight_layout()
    plt.savefig('metrics_heatmap.png')
    plt.close()

# Função para plotar o ranking de cada métrica separadamente
def plot_individual_metric_rankings(df, highlight_model):
    metrics = ['mae', 'mape', 'smape', 'rmse', 'training_time']
    for metric in metrics:
        # Ordenar o DataFrame pelo valor da métrica em ordem ascendente
        df_sorted = df.sort_values(by=metric)
        
        plt.figure(figsize=(12, 10))
        ax = sns.barplot(x=metric, y='model', data=df_sorted, orient='h')
        plt.title(f'Ranking - {metric.upper()}')
        plt.xlabel(metric.upper())
        plt.ylabel('Modelo')
        
        # Adicionar valores das métricas ao lado de cada barra
        for i, v in enumerate(df_sorted[metric]):
            ax.text(v, i, f' {v:.4f}', va='center')
        
        # Destacar a barra do modelo específico
        highlight_index = df_sorted[df_sorted['covariates'] == highlight_model].index[0]
        ax.patches[highlight_index].set_facecolor('red')
        
        plt.tight_layout()
        plt.savefig(f'{metric}_ranking.png')
        plt.close()

# Função principal de análise
def analyze_time_series_models(file_path, highlight_model):
    # Carregar e preparar os dados
    df = load_data(file_path)
    
    # Plotar gráficos
    plot_metrics_heatmap(df, highlight_model)
    plot_individual_metric_rankings(df, highlight_model)
    
    # Imprimir resultados
    print("Análise de Modelos de Séries Temporais")
    print("======================================")
    
    metrics = ['mae', 'mape', 'smape', 'rmse', 'training_time']
    
    for metric in metrics:
        best_model = df.loc[df[metric].idxmin()]
        print(f"\nMelhor modelo para {metric.upper()}:")
        print(f"Modelo: {best_model['model']}")
        print(f"Covariáveis: {best_model['covariates']}")
        print(f"Valor: {best_model[metric]:.4f}")
        
        # Adicionar informação sobre o modelo destacado
        highlight_model_data = df[df['covariates'] == highlight_model].iloc[0]
        print(f"\nModelo destacado ({highlight_model}):")
        print(f"Valor: {highlight_model_data[metric]:.4f}")
        print(f"Ranking: {df[metric].rank().loc[highlight_model_data.name]:.0f}")
    
    print("\nRanking completo dos modelos para cada métrica:")
    for metric in metrics:
        print(f"\n{metric.upper()}:")
        print(df[['model', metric]].sort_values(metric).to_string(index=False))
    
    print("\nGráficos gerados:")
    print("- metrics_heatmap.png: Heatmap das métricas de avaliação")
    for metric in metrics:
        print(f"- {metric}_ranking.png: Ranking individual para {metric}")

In [3]:
file_path = "metrics_by_model_combination_filtered.csv"  # Ajuste o nome do arquivo conforme necessário
highlight_model = "dayofweek_num_holiday_hour_month"
analyze_time_series_models(file_path, highlight_model)

Análise de Modelos de Séries Temporais

Melhor modelo para MAE:
Modelo: dayofweek_num_holiday_hour_month_precipType
Covariáveis: dayofweek_num_holiday_hour_month_precipType
Valor: 0.2022

Modelo destacado (dayofweek_num_holiday_hour_month):
Valor: 0.2032
Ranking: 16

Melhor modelo para MAPE:
Modelo: dayofweek_num_holiday_hour_month_precipType
Covariáveis: dayofweek_num_holiday_hour_month_precipType
Valor: 1.0849

Modelo destacado (dayofweek_num_holiday_hour_month):
Valor: 1.1015
Ranking: 24

Melhor modelo para SMAPE:
Modelo: dayofweek_num_holiday_hour_month_precipType
Covariáveis: dayofweek_num_holiday_hour_month_precipType
Valor: 0.2912

Modelo destacado (dayofweek_num_holiday_hour_month):
Valor: 0.2920
Ranking: 16

Melhor modelo para RMSE:
Modelo: bool_weather_missing_values_dayofweek_num_holiday_hour_month_precipType
Covariáveis: bool_weather_missing_values_dayofweek_num_holiday_hour_month_precipType
Valor: 0.2868

Modelo destacado (dayofweek_num_holiday_hour_month):
Valor: 0.2881
R

In [4]:
import itertools
covariates = ['holiday', 'month', 'hour', 'dayofweek_num', 'precipType', 'bool_weather_missing_values']
covariate_combinations = list(itertools.combinations(covariates, 4))

In [5]:
covariate_combinations

[('holiday', 'month', 'hour', 'dayofweek_num'),
 ('holiday', 'month', 'hour', 'precipType'),
 ('holiday', 'month', 'hour', 'bool_weather_missing_values'),
 ('holiday', 'month', 'dayofweek_num', 'precipType'),
 ('holiday', 'month', 'dayofweek_num', 'bool_weather_missing_values'),
 ('holiday', 'month', 'precipType', 'bool_weather_missing_values'),
 ('holiday', 'hour', 'dayofweek_num', 'precipType'),
 ('holiday', 'hour', 'dayofweek_num', 'bool_weather_missing_values'),
 ('holiday', 'hour', 'precipType', 'bool_weather_missing_values'),
 ('holiday', 'dayofweek_num', 'precipType', 'bool_weather_missing_values'),
 ('month', 'hour', 'dayofweek_num', 'precipType'),
 ('month', 'hour', 'dayofweek_num', 'bool_weather_missing_values'),
 ('month', 'hour', 'precipType', 'bool_weather_missing_values'),
 ('month', 'dayofweek_num', 'precipType', 'bool_weather_missing_values'),
 ('hour', 'dayofweek_num', 'precipType', 'bool_weather_missing_values')]